In [1]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 999

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
offline.init_notebook_mode(connected=True)
import plotly.express as px

from mpl_toolkits.axes_grid1 import make_axes_locatable
# %pylab inline
# pylab.rcParams['figure.figsize'] = (12, 12)
import seaborn as sns



# For changes in .py
# %reload_ext autoreload
# %autoreload 2

import numpy as np

import missingno as msno

stats = ['skew', 'mad', 'kurt']


import qgrid

def qg(df):
    return(qgrid.show_grid(df,show_toolbar=True, grid_options={'forceFitColumns': False}))

import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', -1)
pd.set_option('display.max_colwidth', 100)

import requests
import json
import xmltodict
import re
import urllib.request as urllib2
import io
import yaml

import math
import pysal as ps

# from pysal.esda.mapclassify import Quantiles, Equal_Interval, Fisher_Jenks



import imageio


from sklearn import preprocessing


In [2]:
def clean(df, ano, g_cols):
    df = df.drop(columns=['Unnamed: 16'])
    df.columns = pd.Series(df.columns).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace(' ','_').str.lower().str.strip().tolist()    
    mask = df['orgao'].notnull()
    df = df[mask]
    
    for col in df.columns[-6:]:
        df[col] = df[col].str.replace('.','').str.replace(',','.')
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    mask = df['dotacao_atual']!=0
    df = df[mask]
    
    values = [
                '003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES',
                '002 - RECURSOS VINCULADOS ESTADUAIS',
                '005 - RECURSOS VINCULADOS FEDERAIS',
                '045 - REC.VINC.TRANSF.FEDERAL/SUPERAVIT FINANC.',
                '007 - OP.CRED.E CONTRIB.DO EXTERIOR-DOT.INIC.CR.SU',
                '042 - REC.VINC.ESTADUAIS-CRED.SUPERAVIT FINANCEIRO'
    ]

    for value in values:
        mask = df['fonte_de_recursos']!=value
        df = df[mask]
    
    
    df['sobras'] = df['dotacao_atual'] - df['empenhado'] 
#     cols = ['dotacao_atual','empenhado','sobras']
    df = df.sort_values(by='sobras', ascending=False)
    
    
    df = df.groupby(by=g_cols, as_index=False).sum().sort_values(by=['sobras','dotacao_atual','empenhado'], ascending=False)
    df = df[g_cols + ['sobras','dotacao_atual','empenhado']]
    df['ano'] = ano
    
    cols = ['ano'] + g_cols + ['dotacao_atual','empenhado','sobras']
    for col in g_cols:
        df[col]=df[col].str.strip()
        
        
    return df[cols]

In [3]:
years = [str(i) for i in range(2015,2020)]

# m_cols = g_cols
# m_cols.remove('orgao')
df_final = pd.DataFrame()
for year in years:
    df_year = pd.read_csv('../data/orcamento/orcamento_{}.csv'.format(year), encoding="ISO-8859-9")
    g_cols = ['orgao','fonte_de_recursos','funcao','acao']
    df_year = clean(df_year,year,g_cols)
    df_final = pd.concat([df_final,df_year],axis=0)


In [4]:
df = df_final.sort_values(by=['acao','ano'])
df = df.sort_values(by='sobras', ascending=False)

In [10]:
df.head()

,ano,orgao,fonte_de_recursos,funcao,acao,dotacao_atual,empenhado,sobras
398,2016,21000 - ADMINISTRACAO GERAL DO ESTADO,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,28 - ENCARGOS ESPECIAIS,51400000 - PAGAMENTO DA DIVIDA PUBLICA INTERNA,1.614510e+10,9.109301e+09,7.035797e+09
418,2015,21000 - ADMINISTRACAO GERAL DO ESTADO,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,28 - ENCARGOS ESPECIAIS,50290000 - PAGAMENTO DE SERVICOS GERAIS DO ESTADO,2.939080e+09,7.021884e+08,2.236892e+09
450,2019,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,081 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR-INTRA,09 - PREVIDENCIA SOCIAL,57530000 - GESTAO DOS BENEFICIOS PREVIDENCIARIOS,2.939659e+10,2.886218e+10,5.344076e+08
386,2018,20000 - SECRETARIA DA FAZENDA E PLANEJAMENTO,004 - REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.,09 - PREVIDENCIA SOCIAL,57530000 - CONCESSAO E PAGAMENTO DE BENEFICIOS,3.232790e+10,3.180617e+10,5.217329e+08
43,2015,08000 - SECRETARIA DA EDUCACAO,004 - REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.,12 - EDUCACAO,24940000 - CONSTRUCAO E AMPLIACAO DA REDE FISICA ESCOLAR,5.669286e+08,2.990353e+08,2.678933e+08


### Orçamento 2020

In [23]:
df_year = pd.read_csv('../data/orcamento/orcamento_2020.csv'.format(year), encoding="ISO-8859-9")

In [28]:
mask = df_year['Órgão'].notnull()
df_year = df_year[mask]

In [30]:
mask = df_year['Ação'].str.contains("PUBLICIDADE")


publicidade  = df_year[mask]

In [33]:
publicidade.head()

,Órgão,UO,Unidade Gestora,Fonte de Recursos,Função,Sub Função,Programa,Ação,Funcional Programática,Elemento,Dotação Inicial,Dotação Atual,Empenhado,Liquidado,Pago,Pago Restos,Unnamed: 16
17,01000 - ASSEMBLEIA LEGISLATIVA,01001 - ASSEMBLEIA LEGISLATIVA,010010 - ASSEMBLEIA LEGISLATIVA,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,01 - LEGISLATIVA,031 - ACAO LEGISLATIVA,0150 - PROCESSO LEGISLATIVO,62750000 - PUBLICIDADE INSTITUCIONAL DA ALESP,01031015062750000 - PUBLICIDADE INSTITUCIONAL DA ALESP,339039 - OUTROS SERVICOS DE TERCEIROS-PESSOA JURIDICA,"10,00","0,00","0,00","0,00","0,00","0,00",NaN
50,01000 - ASSEMBLEIA LEGISLATIVA,01001 - ASSEMBLEIA LEGISLATIVA,010101 - ASSEMBLEIA LEGISLATIVA,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,01 - LEGISLATIVA,031 - ACAO LEGISLATIVA,0150 - PROCESSO LEGISLATIVO,62750000 - PUBLICIDADE INSTITUCIONAL DA ALESP,01031015062750000 - PUBLICIDADE INSTITUCIONAL DA ALESP,339039 - OUTROS SERVICOS DE TERCEIROS-PESSOA JURIDICA,"0,00","10,00","0,00","0,00","0,00","0,00",NaN
304,08000 - SECRETARIA DA EDUCACAO,08001 - ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,080010 - ADM. SUP. DA SECRETARIA E DA SEDE,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,12 - EDUCACAO,131 - COMUNICACAO SOCIAL,5123 - COMUNICACAO SOCIAL,55760000 - PUBLICIDADE DE UTILIDADE PUBLICA,12131512355760000 - PUBLICIDADE DE UTILIDADE PUBLICA,339039 - OUTROS SERVICOS DE TERCEIROS-PESSOA JURIDICA,"19.107.900,00","19.107.900,00","0,00","0,00","0,00","0,00",NaN
410,08000 - SECRETARIA DA EDUCACAO,08001 - ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,080101 - GABINETE DO SECRETARIO,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,12 - EDUCACAO,131 - COMUNICACAO SOCIAL,2826 - COMUNICACAO SOCIAL,55760000 - PUBLICIDADE DE UTILIDADE PUBLICA,12131282655760000 - PUBLICIDADE DE UTILIDADE PUBLICA,339039 - OUTROS SERVICOS DE TERCEIROS-PESSOA JURIDICA,"0,00","0,00","0,00","0,00","0,00","2.535.022,24",NaN
4632,09000 - SECRETARIA DA SAUDE,09001 - ADMINISTRACAO SUPERIOR SECRETARIA E SEDE,090010 - ADMIN. SUPERIOR DA SEC. E DA SEDE,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,10 - SAUDE,131 - COMUNICACAO SOCIAL,5123 - COMUNICACAO SOCIAL,55760000 - PUBLICIDADE DE UTILIDADE PUBLICA,10131512355760000 - PUBLICIDADE DE UTILIDADE PUBLICA,339039 - OUTROS SERVICOS DE TERCEIROS-PESSOA JURIDICA,"0,00","24.319.488,00","0,00","0,00","0,00","0,00",NaN


In [10]:
# df.to_excel('sobras_orcamentarias_2015_a_2019.xlsx', index=False, encoding='utf-8')